<a href="https://colab.research.google.com/github/nisha1365/Playground-Series-S3E12-Kaggle-competition/blob/main/Playground_Series_Season_3%2C_Episode_12_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [203]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from itertools import product

In [204]:
train= pd.read_csv("/content/train.csv").reset_index(drop=True)

In [205]:
train.head()

,id,gravity,ph,osmo,cond,urea,calc,target
0,0,1.013,6.19,443,14.8,124,1.45,0
1,1,1.025,5.40,703,23.6,394,4.18,0
2,2,1.009,6.13,371,24.5,159,9.04,0
3,3,1.021,4.91,442,20.8,398,6.63,1
4,4,1.021,5.53,874,17.8,385,2.21,1


In [206]:
X = train.drop(columns="target")
y = train["target"]

In [207]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

In [221]:
X_test = pd.read_csv('/content/test.csv')

In [222]:
X_test.head()

,id,gravity,ph,osmo,cond,urea,calc
0,414,1.017,5.24,345,11.5,152,1.16
1,415,1.020,5.68,874,29.0,385,3.46
2,416,1.024,5.36,698,19.5,354,13.00
3,417,1.020,5.33,668,25.3,252,3.46
4,418,1.011,5.87,567,29.0,457,2.36


In [223]:
search_space = {'fit_intercept': [True, False],
                'intercept_scaling': [1], 'random_state': [42], 'max_iter' : [150], 'n_jobs' : [4]}

In [224]:
min_score = 0
best_params = {}

for val in product(*search_space.values()):
    params = {}
    for i, param in enumerate(search_space.keys()):
        params[param] = val[i]
    print(params)
    clf = LogisticRegression(**params).fit(X_train,y_train)
    val_pred=clf.predict_proba(X_val)[:,1]
    score = roc_auc_score(y_val, val_pred)
    if score > min_score:
        min_score = score
        best_params = params
        print(f'Best score: {score}')

{'fit_intercept': True, 'intercept_scaling': 1, 'random_state': 42, 'max_iter': 150, 'n_jobs': 4}
Best score: 0.8316129032258064
{'fit_intercept': False, 'intercept_scaling': 1, 'random_state': 42, 'max_iter': 150, 'n_jobs': 4}
Best score: 0.8376344086021505


In [225]:
best_params

{'fit_intercept': False,
 'intercept_scaling': 1,
 'random_state': 42,
 'max_iter': 150,
 'n_jobs': 4}

In [226]:
min_score

0.8376344086021505

In [227]:
clf = LogisticRegression(**best_params)

In [228]:
clf.fit(X_train, y_train)

LogisticRegression(fit_intercept=False, max_iter=150, n_jobs=4, random_state=42)

In [229]:
train_pred = clf.predict_proba(X_train)[:,1]

In [230]:
roc_auc_score(y_train, train_pred)

0.7658381808566896

In [231]:
val_pred = clf.predict_proba(X_val)[:,1]

In [232]:
score = roc_auc_score(y_val, val_pred)
score

0.8376344086021505

In [233]:
submission = pd.DataFrame({'id':X_test.index
                          ,'target':clf.predict_proba(X_test)[:,1]})
submission

,id,target
0,0,0.273744
1,1,0.463379
2,2,0.926008
3,3,0.452316
4,4,0.353785
...,...,...
271,271,0.732224
272,272,0.318958
273,273,0.569537
274,274,0.346771


In [234]:
submission.to_csv('submission.csv', index=False)